## **RECOMMENDATION SYSTEM 1: Using Neural Collaborative Filtering (NCF) Model**

Install and Import Required Libraries:

In [39]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs


Read the Data:

In [40]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])


Build Vocabularies:

In [41]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)


Define the NCF Model:

In [42]:
class NCFModel(tfrs.Model):
    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        return self.task(user_embeddings, movie_embeddings)


Define User and Movie Models with NCF Architecture:

In [43]:
embedding_dim = 64

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), embedding_dim),
    tf.keras.layers.Dense(embedding_dim, activation='relu')
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), embedding_dim),
    tf.keras.layers.Dense(embedding_dim, activation='relu')
])


Define the Retrieval Task:

In [44]:
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        movies.batch(128).map(movie_model)
    )
)


Compile and Train the Model:

In [45]:
# Create a retrieval model.
model = NCFModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)


Epoch 1/3
25/25 [==============================] - 23s 845ms/step - factorized_top_k/top_1_categorical_accuracy: 0.6304 - factorized_top_k/top_5_categorical_accuracy: 0.6349 - factorized_top_k/top_10_categorical_accuracy: 0.6374 - factorized_top_k/top_50_categorical_accuracy: 0.6466 - factorized_top_k/top_100_categorical_accuracy: 0.6542 - loss: 609585.7176 - regularization_loss: 0.0000e+00 - total_loss: 609585.7176
Epoch 2/3
25/25 [==============================] - 8s 334ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9530 - factorized_top_k/top_5_categorical_accuracy: 0.9535 - factorized_top_k/top_10_categorical_accuracy: 0.9537 - factorized_top_k/top_50_categorical_accuracy: 0.9551 - factorized_top_k/top_100_categorical_accuracy: 0.9566 - loss: 32460.4747 - regularization_loss: 0.0000e+00 - total_loss: 32460.4747
Epoch 3/3
25/25 [==============================] - 9s 346ms/step - factorized_top_k/top_1_categorical_accuracy: 0.9644 - factorized_top_k/top_5_categorical_accurac

Evaluate and Get Recommendations:

In [46]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print("Top 5 recommendations for user 42:")
for i, title in enumerate(titles[0, :5], 1):
    print(f"{i}. {title.numpy().decode('utf-8')}")


Top 5 recommendations for user 42:
1. You So Crazy (1994)
2. Love Is All There Is (1996)
3. Fly Away Home (1996)
4. In the Line of Duty 2 (1987)
5. Niagara, Niagara (1997)


## **RECOMMENDATION SYSTEM 2: Using Bayesian Personalized Ranking (BPR) Model**

Install and Import Required Libraries:

In [47]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs


Read the Data:

In [48]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])


Build Vocabularies:

In [49]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)


Define the BPR Model:

In [50]:
class BPRModel(tfrs.Model):
    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        positive_movie_embeddings = self.movie_model(features["movie_title"])

        # Generate negative samples
        negative_movie_embeddings = self.movie_model(tf.random.shuffle(features["movie_title"]))

        # Calculate the BPR loss
        positive_scores = tf.reduce_sum(user_embeddings * positive_movie_embeddings, axis=1)
        negative_scores = tf.reduce_sum(user_embeddings * negative_movie_embeddings, axis=1)
        bpr_loss = -tf.reduce_mean(tf.math.log(tf.nn.sigmoid(positive_scores - negative_scores)))

        return self.task(user_embeddings, positive_movie_embeddings) + bpr_loss


Define User and Movie Models:





In [51]:
embedding_dim = 64

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), embedding_dim)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), embedding_dim)
])


Define the Retrieval Task:

In [52]:
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        movies.batch(128).map(movie_model)
    )
)


Compile and Train the Model:

In [53]:
# Create a retrieval model.
model = BPRModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)


Epoch 1/3
25/25 [==============================] - 25s 984ms/step - factorized_top_k/top_1_categorical_accuracy: 7.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0014 - factorized_top_k/top_10_categorical_accuracy: 0.0046 - factorized_top_k/top_50_categorical_accuracy: 0.0436 - factorized_top_k/top_100_categorical_accuracy: 0.0979 - loss: 33102.0699 - regularization_loss: 0.0000e+00 - total_loss: 33102.0699
Epoch 2/3
25/25 [==============================] - 15s 580ms/step - factorized_top_k/top_1_categorical_accuracy: 1.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0049 - factorized_top_k/top_10_categorical_accuracy: 0.0140 - factorized_top_k/top_50_categorical_accuracy: 0.1056 - factorized_top_k/top_100_categorical_accuracy: 0.2109 - loss: 31011.8137 - regularization_loss: 0.0000e+00 - total_loss: 31011.8137
Epoch 3/3
25/25 [==============================] - 15s 588ms/step - factorized_top_k/top_1_categorical_accuracy: 4.1000e-04 - factorized_top_k/top_5_categor

Evaluate and Get Recommendations:

In [54]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print("Top 5 recommendations for user 42:")
for i, title in enumerate(titles[0, :5], 1):
    print(f"{i}. {title.numpy().decode('utf-8')}")


Top 5 recommendations for user 42:
1. Rent-a-Kid (1995)
2. Aristocats, The (1970)
3. Just Cause (1995)
4. House Arrest (1996)
5. Winnie the Pooh and the Blustery Day (1968)


## **RECOMMENDATION SYSTEM 3: Using Graph Neural Network (GNN) Model**

Install and Import Required Libraries:

In [55]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs


Read the Data:

In [56]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])


Build Vocabularies:

In [57]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)


Define the GNN Model:

In [58]:
class GNNModel(tfrs.Model):
    def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        return self.task(user_embeddings, movie_embeddings)


Define User and Movie Models with GNN Layers:








In [59]:
embedding_dim = 64

class GNNLayer(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.weight = self.add_weight(shape=(input_shape[-1], self.units), initializer="random_normal", trainable=True)
        self.bias = self.add_weight(shape=(self.units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.nn.relu(tf.matmul(inputs, self.weight) + self.bias)

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), embedding_dim),
    GNNLayer(embedding_dim)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), embedding_dim),
    GNNLayer(embedding_dim)
])


Define the Retrieval Task:

In [60]:
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        movies.batch(128).map(movie_model)
    )
)


Compile and Train the Model:

In [61]:
# Create a retrieval model.
model = GNNModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)


Epoch 1/3
25/25 [==============================] - 24s 891ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1209 - factorized_top_k/top_5_categorical_accuracy: 0.3009 - factorized_top_k/top_10_categorical_accuracy: 0.3086 - factorized_top_k/top_50_categorical_accuracy: 0.3783 - factorized_top_k/top_100_categorical_accuracy: 0.4281 - loss: 923564.4526 - regularization_loss: 0.0000e+00 - total_loss: 923564.4526
Epoch 2/3
25/25 [==============================] - 10s 412ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2695 - factorized_top_k/top_5_categorical_accuracy: 0.5209 - factorized_top_k/top_10_categorical_accuracy: 0.5253 - factorized_top_k/top_50_categorical_accuracy: 0.6866 - factorized_top_k/top_100_categorical_accuracy: 0.7719 - loss: 33011.7733 - regularization_loss: 0.0000e+00 - total_loss: 33011.7733
Epoch 3/3
25/25 [==============================] - 10s 390ms/step - factorized_top_k/top_1_categorical_accuracy: 0.5879 - factorized_top_k/top_5_categorical_accur

Evaluate and Get Recommendations:

In [62]:
# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print("Top 5 recommendations for user 42:")
for i, title in enumerate(titles[0, :5], 1):
    print(f"{i}. {title.numpy().decode('utf-8')}")


Top 5 recommendations for user 42:
1. Star Trek: First Contact (1996)
2. Herbie Rides Again (1974)
3. Life Less Ordinary, A (1997)
4. Lion King, The (1994)
5. Last Time I Saw Paris, The (1954)
